In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import mlflow
import plotly.graph_objects as go
import plotly.express as px
import mlflow.sklearn
import mlflow.xgboost
import xgboost as xgb
import optuna
import os

from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from sksurv.metrics import concordance_index_censored

/home/joelt/miniconda3/envs/projet-business/lib/python3.9/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/home/joelt/miniconda3/envs/projet-business/lib/python3.9/site-packages/google/oauth2/__init__.py:40: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/home/joelt/miniconda3/envs/projet-business/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See h

In [2]:
# Test GPU par XGBoost
try:
    # On crée une micro-matrice de test
    data = xgb.DMatrix([[1, 2], [3, 4]], label=[1, 0])

    params = {'tree_method': 'gpu_hist', 'device': 'cuda'}
    xgb.train(params, data, num_boost_round=1)
    print("✅ Succès ! La RTX 4060 est reconnue et configurée.")
except Exception as e:
    print(f"❌ Échec du GPU : {e}")
    print("Le modèle tournera sur CPU par défaut.")

✅ Succès ! La RTX 4060 est reconnue et configurée.


/home/joelt/miniconda3/envs/projet-business/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:06:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [3]:
df = pd.read_parquet('dataset_full.parquet')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Structure du dataset : {df.shape[0]} lignes et {df.shape[1]} colonnes")
display(df.head())

Structure du dataset : 1202455 lignes et 17 colonnes


,SIREN,Code postal de l'établissement,Code commune de l'établissement,Dénomination de l'unité légale,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Economie sociale et solidaire unité légale,Code du département de l'établissement,Code de la région de l'établissement,Date_fermeture_finale,Tranche_effectif_num,age_estime,latitude,longitude,code_ape,libelle_section_ape,fermeture
0,912239670,26740,26191,LES TOITURES PERENNES,5499,43.91A,N,26,84,2025-12-31,3,4,44.5480,4.7964,43,Travaux de construction spécialisés,0
1,389091893,91270,91657,OCB,5499,43.99C,N,91,11,2025-12-31,10,33,48.7156,2.4344,43,Travaux de construction spécialisés,0
2,888366747,30310,30344,COMME UN DIADEME,5710,47.91A,N,30,76,2025-12-31,1,5,43.7432,4.2062,47,"Commerce de détail, à l’exception des automobi...",0
3,953937455,75008,75108,WE CLEAN HOLDING PARNASSA,5710,66.30Z,N,75,11,2025-12-31,0,2,48.8731,2.3166,66,Activités auxiliaires de services financiers e...,0
4,840236111,94440,94070,LEVIO,5710,96.02A,N,94,11,2025-12-31,1,8,48.7283,2.5747,96,Autres services personnels,0


In [4]:
# --- AJUSTEMENTS POUR LE MODÈLE SURVIVAL AFT ---

# 1. NETTOYAGE GÉOGRAPHIQUE
df['Code du département de l\'établissement'] = df['Code du département de l\'établissement'].astype(str).str.zfill(2)
dep_risk_map = df.groupby("Code du département de l'établissement")["fermeture"].mean()
df['risque_departemental'] = df['Code du département de l\'établissement'].map(dep_risk_map)

# 2. TRAITEMENT DES TYPES
df['Catégorie juridique de l\'unité légale'] = df['Catégorie juridique de l\'unité légale'].astype(str)
df['age_estime'] = df['age_estime'].astype(float)
df['Tranche_effectif_num'] = df['Tranche_effectif_num'].fillna(0).astype(float)

# 3. ENCODAGE DES VARIABLES

df['is_ess'] = df['Economie sociale et solidaire unité légale'].map({'O': 1, 'N': 0}).fillna(0).astype(int)

# B. One-Hot Encoding

df_final = pd.get_dummies(
    df, 
    columns=['libelle_section_ape', 'Catégorie juridique de l\'unité légale'], 
    prefix=['APE', 'CJ'],
    drop_first=True,
    dtype=int 
)

# 4. SÉLECTION FINALE

cols_to_drop = [
    'Code postal de l\'établissement', 'Code commune de l\'établissement',
    'Activité principale de l\'unité légale', 'Date_fermeture_finale', 
    'latitude', 'longitude', 'code_ape',
    'Code du département de l\'établissement', 'Code de la région de l\'établissement',
    'Economie sociale et solidaire unité légale'
]
df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns])

# 5. DERNIERS RÉGLAGES POUR LA SURVIE (Crucial pour AFT)

df_final = df_final[df_final['age_estime'] > 0].copy()

# On crée les colonnes cibles pour le modèle AFT
# y_lower : l'âge au dernier moment où on sait que l'entreprise est en vie
# y_upper : l'âge au moment du décès (ou +inf si toujours vivante)
df_final['y_lower'] = df_final['age_estime']
df_final['y_upper'] = np.where(df_final['fermeture'] == 1, df_final['age_estime'], np.inf)

print(f"✅ Dataset finalisé : {df_final.shape[0]} lignes, {df_final.shape[1]} colonnes.")

✅ Dataset finalisé : 1195740 lignes, 94 colonnes.


In [5]:
# --- 1. ANALYSE DES FRÉQUENCES ---
binary_cols = [c for c in df_final.columns if c.startswith('APE_') or c.startswith('CJ_')]
frequencies = df_final[binary_cols].mean().sort_values(ascending=False) * 100

# Définition du seuil (0.1%)
rare_limit = 0.1
rare_cols = frequencies[frequencies < rare_limit]

print(f"--- 🔍 Analyse des colonnes rares (< {rare_limit}%) ---")
print(f"Il y a {len(rare_cols)} colonnes concernées.")

# --- 2. FUSION DES CATÉGORIES RARES ---
# On identifie les colonnes à fusionner à partir de rare_cols qu'on vient de créer
rare_ape_cols = [c for c in rare_cols.index if c.startswith('APE_')]
rare_cj_cols = [c for c in rare_cols.index if c.startswith('CJ_')]

# On crée la colonne "Autres" et on supprime les anciennes
if rare_ape_cols:
    df_final['APE_Autres_Secteurs'] = df_final[rare_ape_cols].any(axis=1).astype(int)
    df_final.drop(columns=rare_ape_cols, inplace=True)

if rare_cj_cols:
    df_final['CJ_Autres_Status'] = df_final[rare_cj_cols].any(axis=1).astype(int)
    df_final.drop(columns=rare_cj_cols, inplace=True)

# --- 3. PRÉPARATION DES CIBLES DE SURVIE (AFT) ---

df_final['y_lower'] = df_final['age_estime']
df_final['y_upper'] = np.where(df_final['fermeture'] == 1, df_final['age_estime'], np.inf)

print(f"✅ Nettoyage et préparation AFT terminés.")
print(f"📊 Nouveau nombre de colonnes : {len(df_final.columns)}")
display(df_final[['age_estime', 'fermeture', 'y_lower', 'y_upper']].head())

--- 🔍 Analyse des colonnes rares (< 0.1%) ---
Il y a 34 colonnes concernées.
✅ Nettoyage et préparation AFT terminés.
📊 Nouveau nombre de colonnes : 61


,age_estime,fermeture,y_lower,y_upper
0,4.0000,0,4.0000,inf
1,33.0000,0,33.0000,inf
2,5.0000,0,5.0000,inf
3,2.0000,0,2.0000,inf
4,8.0000,0,8.0000,inf


##### Train/test split

In [ ]:
# 1. On s'assure d'avoir la colonne d'âge
if 'age_au_diagnostic' not in df_final.columns:
    df_final['age_au_diagnostic'] = df_final['age_estime']

# 2. Définition des colonnes à exclure (on garde l'âge !)
# Assure-toi que 'risque_departemental' est BIEN DANS to_drop si tu veux l'enlever
non_numeric_cols = df_final.select_dtypes(exclude=[np.number]).columns.tolist()
targets = ['fermeture', 'age_estime', 'y_lower', 'y_upper']
to_drop = list(set(non_numeric_cols + targets))

if 'age_au_diagnostic' in to_drop:
    to_drop.remove('age_au_diagnostic')

# --- OPTIONNEL : Si tu veux enlever le risque départemental pour rééquilibrer ---
# to_drop.append('risque_departemental') 

X = df_final.drop(columns=to_drop)
y_time = df_final['age_estime']
y_event = df_final['fermeture'].astype(int)

# 3. Split Train/Test
X_train, X_test, y_train_time, y_test_time, y_train_event, y_test_event = train_test_split(
    X, y_time, y_event, test_size=0.2, random_state=42
)

# 4. Création des DMatrix (Le cerveau d'XGBoost)
# Pour le train
y_upper_train = np.where(y_train_event == 1, y_train_time, np.inf)
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train_time.values)
dtrain.set_float_info('label_upper_bound', y_upper_train)

# Pour le test (Optuna en a besoin pour évaluer)
y_upper_test = np.where(y_test_event == 1, y_test_time, np.inf)
dtest_optuna = xgb.DMatrix(X_test)
dtest_optuna.set_float_info('label_lower_bound', y_test_time.values)
dtest_optuna.set_float_info('label_upper_bound', y_upper_test)

print(f"✅ Données prêtes ! Features utilisées : {X.columns.tolist()}")

✅ Données prêtes ! Features utilisées : ['Tranche_effectif_num', 'risque_departemental', 'is_ess', 'APE_Activités administratives et autres activités de soutien aux entreprises', "APE_Activités auxiliaires de services financiers et d'assurance ", 'APE_Activités créatives, artistiques et de spectacle ', "APE_Activités d'architecture et d'ingénierie ; activités de contrôle et analyses techniques", 'APE_Activités de location et location-bail', 'APE_Activités des agences de voyage, voyagistes, services de réservation et activités connexes', 'APE_Activités des services financiers, hors assurance et caisses de retraite', 'APE_Activités des sièges sociaux ; conseil de gestion', 'APE_Activités immobilières', 'APE_Activités juridiques et comptables', "APE_Activités liées à l'emploi", 'APE_Activités pour la santé humaine', 'APE_Activités sportives, récréatives et de loisirs', 'APE_Autres activités spécialisées, scientifiques et techniques', 'APE_Autres industries manufacturières', 'APE_Autres se

##### Train dans Mlflow

In [ ]:
# --- 1. GESTION DE L'EXPERIMENT MLFLOW ---
# On définit un nouveau nom d'expérience pour éviter de taper dans l'ID 0 supprimé
experiment_name = "XGBoost_Survival_V3_Equilibre"

try:
    # On crée l'expérience
    exp_id = mlflow.create_experiment(experiment_name)
except:
    # Si elle existe déjà, on récupère son ID (pourvu qu'elle ne soit pas 'deleted')
    exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

mlflow.set_experiment(experiment_name)

# --- 2. DÉFINITION DE L'OBJECTIF ---
def objective(trial):
    params = {
        'objective': 'survival:aft',
        'eval_metric': 'aft-nloglik',
        'tree_method': 'hist',
        'device': 'cuda',
        
        # --- RÉGLAGES POUR LA NUANCE (Anti-Binaire) ---
        'max_depth': trial.suggest_int('max_depth', 3, 6), 
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.02, log=True),
        
        # Force des feuilles plus denses pour éviter le 0% de risque brutal
        'min_child_weight': trial.suggest_int('min_child_weight', 10, 50),
        
        # Régularisation forte pour contrer la domination du département
        'lambda': trial.suggest_float('lambda', 1.0, 50.0, log=True),
        'alpha': trial.suggest_float('alpha', 0.1, 10.0, log=True),
        
        'aft_loss_distribution': 'logistic', 
        'aft_loss_distribution_scale': trial.suggest_float('aft_loss_distribution_scale', 0.8, 1.5),
        
        # On réduit le sampling pour que chaque arbre voit des sous-ensembles différents
        'subsample': trial.suggest_float('subsample', 0.5, 0.8),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
    }

    # Un run "nested" (imbriqué) pour chaque essai Optuna
    with mlflow.start_run(nested=True):
        bst_trial = xgb.train(
            params, 
            dtrain, 
            num_boost_round=2000, 
            evals=[(dtest_optuna, 'test')],
            early_stopping_rounds=100,
            verbose_eval=False
        )
        
        score = bst_trial.best_score
        mlflow.log_params(params)
        mlflow.log_metric("test_nloglik", score)
        
        return score

# --- 3. LANCEMENT DE L'OPTIMISATION ---
study = optuna.create_study(direction='minimize')

# On englobe l'étude dans un run parent identifié par l'exp_id
with mlflow.start_run(run_name="Train_V3_Equilibre_Anciennete", experiment_id=exp_id):
    study.optimize(objective, n_trials=50)

# --- 4. RÉCUPÉRATION DU MEILLEUR MODÈLE ---
print(f"✅ Meilleur score : {study.best_value}")
print(f"🏆 Paramètres : {study.best_params}")

[I 2026-02-18 13:10:18,687] A new study created in memory with name: no-name-a323a53f-543e-4d21-8f4d-b1d0fb585664
[I 2026-02-18 13:10:49,364] Trial 0 finished with value: 1.3152242050778666 and parameters: {'max_depth': 3, 'learning_rate': 0.004138516525512685, 'min_child_weight': 14, 'lambda': 5.668946710423125, 'alpha': 8.811838966477302, 'aft_loss_distribution_scale': 0.871655542808035, 'subsample': 0.7291305224912401, 'colsample_bytree': 0.5957068735635047}. Best is trial 0 with value: 1.3152242050778666.


🏃 View run abrasive-moth-259 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/8e0755842276403291e7e45bdd4656bd
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10
🏃 View run grandiose-mole-121 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/cf37ff50efcb49ca95281f0848c3840a
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10


[I 2026-02-18 13:11:21,497] Trial 1 finished with value: 1.4105048953829187 and parameters: {'max_depth': 5, 'learning_rate': 0.01582072574359899, 'min_child_weight': 13, 'lambda': 6.149725929931662, 'alpha': 0.17411381717092253, 'aft_loss_distribution_scale': 1.1602973419243336, 'subsample': 0.6601523898078463, 'colsample_bytree': 0.6712287421044636}. Best is trial 0 with value: 1.3152242050778666.


🏃 View run overjoyed-fowl-106 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/526b96920d7d480ba06627512df02283
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10


[I 2026-02-18 13:11:59,201] Trial 2 finished with value: 1.3568280694568728 and parameters: {'max_depth': 6, 'learning_rate': 0.0029490830498242865, 'min_child_weight': 38, 'lambda': 3.028084932048364, 'alpha': 0.2505654122016298, 'aft_loss_distribution_scale': 0.9903457050061859, 'subsample': 0.6599220254522327, 'colsample_bytree': 0.7834957625844394}. Best is trial 0 with value: 1.3152242050778666.
[I 2026-02-18 13:12:34,904] Trial 3 finished with value: 1.4491196538198703 and parameters: {'max_depth': 5, 'learning_rate': 0.0023425467836582187, 'min_child_weight': 17, 'lambda': 7.732890395346283, 'alpha': 0.35881312832009155, 'aft_loss_distribution_scale': 1.276094208730165, 'subsample': 0.7238013023009426, 'colsample_bytree': 0.7475055106828321}. Best is trial 0 with value: 1.3152242050778666.


...

🏃 View run efficient-hawk-574 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/adc49d90f890426baa477bdad138a76c
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10
🏃 View run colorful-moose-577 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/22440bf8285b41d4a36b806dc5f58f18
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10


[I 2026-02-18 13:34:55,947] Trial 46 finished with value: 1.2970997455885702 and parameters: {'max_depth': 3, 'learning_rate': 0.015015247066328364, 'min_child_weight': 39, 'lambda': 1.0533235506465723, 'alpha': 5.482862252729703, 'aft_loss_distribution_scale': 0.8357157028924883, 'subsample': 0.5023731769852554, 'colsample_bytree': 0.5661337328666034}. Best is trial 33 with value: 1.2823413632047709.
[I 2026-02-18 13:35:28,301] Trial 47 finished with value: 1.4622428852797957 and parameters: {'max_depth': 5, 'learning_rate': 0.008064083504418323, 'min_child_weight': 42, 'lambda': 27.767727940498546, 'alpha': 8.452459953355728, 'aft_loss_distribution_scale': 1.336660111179501, 'subsample': 0.7672208029196161, 'colsample_bytree': 0.7221634239312446}. Best is trial 33 with value: 1.2823413632047709.


🏃 View run amusing-frog-583 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/aa70d9bb3972499ba086617bb564a79e
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10
🏃 View run abrasive-bug-685 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/6f9c9fb8a05b487aa0e487914219c33a
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10


[I 2026-02-18 13:35:59,365] Trial 48 finished with value: 1.283947822425076 and parameters: {'max_depth': 4, 'learning_rate': 0.005931748146651735, 'min_child_weight': 31, 'lambda': 15.879166212072175, 'alpha': 0.11445037982488403, 'aft_loss_distribution_scale': 0.804184607086351, 'subsample': 0.6121886262649928, 'colsample_bytree': 0.542162294563864}. Best is trial 33 with value: 1.2823413632047709.
[I 2026-02-18 13:36:29,222] Trial 49 finished with value: 1.3293894180061918 and parameters: {'max_depth': 3, 'learning_rate': 0.0025198547281259647, 'min_child_weight': 32, 'lambda': 5.333727344574659, 'alpha': 0.1186466995913464, 'aft_loss_distribution_scale': 0.8962068537836706, 'subsample': 0.6450271023342926, 'colsample_bytree': 0.5133226997258169}. Best is trial 33 with value: 1.2823413632047709.


🏃 View run judicious-worm-113 at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/20adbb082e604fe18b4e47c6b175e133
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10
🏃 View run Train_V3_Equilibre_Anciennete at: https://djohell-ml-flow.hf.space/#/experiments/10/runs/ac96ad845ed84cc0805c98ed5ff09a43
🧪 View experiment at: https://djohell-ml-flow.hf.space/#/experiments/10
✅ Meilleur score : 1.2823413632047709
🏆 Paramètres : {'max_depth': 5, 'learning_rate': 0.005995980919546177, 'min_child_weight': 40, 'lambda': 21.43189908501998, 'alpha': 4.436638327016408, 'aft_loss_distribution_scale': 0.8029723618578661, 'subsample': 0.54884842958128, 'colsample_bytree': 0.583881330561382}


---

##### Sauvegarde du modèle

In [10]:
# 1. Récupération des meilleurs paramètres
best_params = study.best_params
best_params.update({
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'tree_method': 'hist',
    'device': 'cuda',
    'aft_loss_distribution': 'logistic'
})

print("🚀 Ré-entraînement du modèle final avec les meilleurs paramètres...")

# 2. Entraînement final (on utilise dtrain qui contient 100% du train)
final_model = xgb.train(
    best_params,
    dtrain,
    num_boost_round=2000 # On peut monter un peu plus pour la version finale
)

# 3. Sauvegarde au format JSON (plus sûr pour ton API)
final_model.save_model("model_survie_V3_final.json")
print("✅ Modèle V3 sauvegardé sous 'model_survie_V3_final.json'")

🚀 Ré-entraînement du modèle final avec les meilleurs paramètres...
✅ Modèle V3 sauvegardé sous 'model_survie_V3_final.json'


---

##### Vérification des features importances sur les données de test

In [ ]:
# --- 1. PERFORMANCE SUR LE TEST SET ---
# On calcule le score final sur les données de test non vues
eval_result = final_model.eval(dtest_optuna)
print(f"📊 Performance finale (Test NLogLik) : {eval_result}")

# --- 2. IMPORTANCE DES VARIABLES ---
# On regarde ce qui a vraiment compté pour le modèle
importance = final_model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'Feature': importance.keys(),
    'Importance': importance.values()
}).sort_values(by='Importance', ascending=False)

print("\n🏆 Top 10 des variables les plus influentes :")
print(importance_df.head(10))

📊 Performance finale (Test NLogLik) : [0]	eval-aft-nloglik:1.36655989967515601

🏆 Top 10 des variables les plus influentes :
                                              Feature  Importance
43                                  age_au_diagnostic   4665.1157
0                                Tranche_effectif_num    834.7848
41                                            CJ_5710    516.6942
31  APE_Production et distribution d'électricité, ...    182.0325
8   APE_Activités des services financiers, hors as...    145.0101
10                         APE_Activités immobilières     84.0674
17  APE_Commerce de détail, à l’exception des auto...     82.3635
4   APE_Activités auxiliaires de services financie...     58.3723
1                                risque_departemental     49.8740
34                                   APE_Restauration     43.3258


In [12]:
# --- 3. PRÉDICTION ET RÉPARTITION ---
# Prédiction du Mu (espérance de vie logarithmique)
preds_mu = final_model.predict(dtest_optuna)
sigma = best_params['aft_loss_distribution_scale']

# On calcule un risque arbitraire à 2 ans (point charnière de ton train précédent)
def calculate_risk(mu, horizon=2, s=sigma):
    z = (np.log(horizon) - mu) / s
    return (1 / (1 + np.exp(-z))) * 100

# Application aux résultats
risques_2ans = [calculate_risk(m) for m in preds_mu]

# Création d'un mini-df d'analyse
analysis_df = pd.DataFrame({
    'mu': preds_mu,
    'risk_2y': risques_2ans,
    'age_reel': X_test['age_au_diagnostic'] # Assure-toi que le nom correspond
})

# Définition des profils (ex: > 50% de risque à 2 ans = Critique)
critique = analysis_df[analysis_df['risk_2y'] > 50]
solide = analysis_df[analysis_df['risk_2y'] <= 10]

print("-" * 50)
print(f"🏢 Entreprises analysées (Test) : {len(analysis_df)}")
print(f"🔴 Portefeuille critique (>50% risque) : {len(critique)} ({len(critique)/len(analysis_df):.1%})")
print(f"🟢 Portefeuille solide (<10% risque) : {len(solide)} ({len(solide)/len(analysis_df):.1%})")
print("-" * 50)
print(f"💡 Âge moyen (Profil Critique) : {critique['age_reel'].mean():.1f} ans")
print(f"👴 Âge moyen (Profil Solide) : {solide['age_reel'].mean():.1f} ans")

/tmp/ipykernel_11880/2853316428.py:9: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-z))) * 100


--------------------------------------------------
🏢 Entreprises analysées (Test) : 239148
🔴 Portefeuille critique (>50% risque) : 0 (0.0%)
🟢 Portefeuille solide (<10% risque) : 233561 (97.7%)
--------------------------------------------------
💡 Âge moyen (Profil Critique) : nan ans
👴 Âge moyen (Profil Solide) : 11.0 ans


In [13]:
def calculate_risk_safe(mu, horizon=5, s=sigma):
    # On utilise np.clip pour éviter que z ne devienne trop grand (overflow)
    z = (np.log(horizon) - mu) / s
    z = np.clip(z, -50, 50) 
    # Logistique robuste
    return (1 / (1 + np.exp(-z))) * 100

# On teste sur un horizon plus lointain (5 ans) pour voir les fragilités
analysis_df['risk_5y'] = analysis_df['mu'].apply(lambda x: calculate_risk_safe(x, horizon=5))

# On baisse un peu le seuil du "critique" pour le diagnostic (ex: 20% à 5 ans)
critique = analysis_df[analysis_df['risk_5y'] > 20]
solide = analysis_df[analysis_df['risk_5y'] <= 5]

print("-" * 50)
print(f"🏢 Entreprises analysées : {len(analysis_df)}")
print(f"🔴 Profil Fragile (>20% risque à 5 ans) : {len(critique)} ({len(critique)/len(analysis_df):.1%})")
print(f"🟢 Profil Très Solide (<5% risque à 5 ans) : {len(solide)} ({len(solide)/len(analysis_df):.1%})")
print("-" * 50)
if len(critique) > 0:
    print(f"💡 Âge moyen (Fragile) : {critique['age_reel'].mean():.1f} ans")
print(f"👴 Âge moyen (Solide) : {solide['age_reel'].mean():.1f} ans")

--------------------------------------------------
🏢 Entreprises analysées : 239148
🔴 Profil Fragile (>20% risque à 5 ans) : 9137 (3.8%)
🟢 Profil Très Solide (<5% risque à 5 ans) : 220466 (92.2%)
--------------------------------------------------
💡 Âge moyen (Fragile) : 1.5 ans
👴 Âge moyen (Solide) : 11.5 ans


##### Vérification des risques du portefeuille

In [ ]:
# 1. Préparation de la matrice globale (X_total)
# On utilise les mêmes colonnes que celles utilisées pour l'entraînement
X_total = df_final[X.columns] 
dmatrix_total = xgb.DMatrix(X_total)

# 2. Prédiction des Mu et calcul des risques
preds_mu_total = final_model.predict(dmatrix_total)
sigma = best_params['aft_loss_distribution_scale']

def get_risk(mu, horizon=5):
    z = (np.log(horizon) - mu) / sigma
    z = np.clip(z, -50, 50)
    return (1 / (1 + np.exp(-z))) * 100

# 3. Création du DataFrame de répartition
ptf_analysis = pd.DataFrame({
    'age': df_final['age_au_diagnostic'],
    'risk_5y': [get_risk(m) for m in preds_mu_total]
})

# 4. Segmentation en échelons (Labels de risque)
bins = [0, 5, 10, 20, 100]
labels = ['🟢 Très Solide', '🟡 Stable', '🟠 Fragile', '🔴 Critique']
ptf_analysis['segment'] = pd.cut(ptf_analysis['risk_5y'], bins=bins, labels=labels)

# 5. Calcul de la répartition
repartition = ptf_analysis.groupby('segment', observed=True).agg(
    nombre_entreprises=('age', 'count'),
    age_moyen=('age', 'mean')
).reset_index()

repartition['pourcentage'] = (repartition['nombre_entreprises'] / len(ptf_analysis) * 100).round(2)

print("📊 RÉPARTITION GLOBALE DU PORTEFEUILLE (Horizon 5 ans)")
print("-" * 60)
print(repartition.to_string(index=False))
print("-" * 60)

📊 RÉPARTITION GLOBALE DU PORTEFEUILLE (Horizon 5 ans)
------------------------------------------------------------
      segment  nombre_entreprises  age_moyen  pourcentage
🟢 Très Solide             1102135    11.5222      92.1700
     🟡 Stable               10469     2.1738       0.8800
    🟠 Fragile               37196     1.9981       3.1100
   🔴 Critique               45940     1.4776       3.8400
------------------------------------------------------------


##### Vérification des secteurs les plus à risques

In [15]:
# Top 5 des secteurs les plus représentés dans le segment Critique
critique_df = df_final.loc[ptf_analysis[ptf_analysis['segment'] == '🔴 Critique'].index]
top_ape_critique = critique_df.filter(like='APE_').sum().sort_values(ascending=False).head(5)

print("🔥 Top 5 des secteurs les plus à risque (Segment Critique) :")
print(top_ape_critique)

🔥 Top 5 des secteurs les plus à risque (Segment Critique) :
APE_Commerce de détail, à l’exception des automobiles et des motocycles    11935
APE_Travaux de construction spécialisés                                     6170
APE_Commerce de gros, à l’exception des automobiles et des motocycles       4524
APE_Restauration                                                            4388
APE_Activités des sièges sociaux ; conseil de gestion                       2206
dtype: int64
